In this notebook we will use a <a href="https://scikit-learn.org/stable/" target="_blank" rel="noopener">`scikit-learn`</a> model created earlier to predict the age of an abalone from its physical measurements and sex. You can find more information about the problem domain <a href="https://archive.ics.uci.edu/dataset/1/abalone" target="_blank" rel="noopener">here</a>.

We will be using a generic prediction UDF script. To execute queries and load data from Exasol database we will be using the <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a> module.

Prior to running this notebook one needs to complete the follow steps:
1. [Create generic scikit-learn prediction UDF script](sklearn_predict_udf.ipynb).
2. [Train a model on the Abalone](sklearn_train_abalone.ipynb).

In [1]:
# TODO: Move this to a separate configuration notebook. Here we just need to load this configuration from a store.
EXASOL_EXTERNAL_HOST_NAME = "192.168.124.93"
EXASOL_HOST_PORT = "8888"
EXASOL_EXTERNAL_HOST = f"""{EXASOL_EXTERNAL_HOST_NAME}:{EXASOL_HOST_PORT}"""
EXASOL_USER = "sys"
EXASOL_PASSWORD = "exasol"
EXASOL_BUCKETFS_PORT = "6666"
EXASOL_EXTERNAL_BUCKETFS_HOST = f"""{EXASOL_EXTERNAL_HOST_NAME}:{EXASOL_BUCKETFS_PORT}"""
EXASOL_BUCKETFS_USER = "w"
EXASOL_BUCKETFS_PASSWORD = "write"
EXASOL_BUCKETFS_USE_HTTPS = False
EXASOL_BUCKETFS_URL_PREFIX = "https://" if EXASOL_BUCKETFS_USE_HTTPS else "http://"
EXASOL_BUCKETFS_SERVICE = "bfsdefault"
EXASOL_BUCKETFS_BUCKET = "default"
EXASOL_BUCKETFS_PATH = f"/buckets/{EXASOL_BUCKETFS_SERVICE}/{EXASOL_BUCKETFS_BUCKET}" # Filesystem-Path to the read-only mounted BucketFS inside the running UDF Container
EXASOL_SCRIPT_LANGUAGE_NAME = "PYTHON3_60"
EXASOL_UDF_FLAVOR = "python3-ds-EXASOL-6.0.0"
EXASOL_UDF_RELEASE= "20190116"
EXASOL_UDF_CLIENT = "exaudfclient" # or for newer versions of the flavor exaudfclient_py3
EXASOL_SCRIPT_LANGUAGES = f"{EXASOL_SCRIPT_LANGUAGE_NAME}=localzmq+protobuf:///{EXASOL_BUCKETFS_SERVICE}/{EXASOL_BUCKETFS_BUCKET}/{EXASOL_UDF_FLAVOR}?lang=python#buckets/{EXASOL_BUCKETFS_SERVICE}/{EXASOL_BUCKETFS_BUCKET}/{EXASOL_UDF_FLAVOR}/exaudf/{EXASOL_UDF_CLIENT}";
EXASOL_SCHEMA = "IDA"

connection_params = {"dns": EXASOL_EXTERNAL_HOST, "user": EXASOL_USER, "password": EXASOL_PASSWORD, "compression": True}

params = {
    "script_languages": EXASOL_SCRIPT_LANGUAGES,
    "script_language_name": EXASOL_SCRIPT_LANGUAGE_NAME,
    "schema": EXASOL_SCHEMA,
    "EXASOL_BUCKETFS_PORT": EXASOL_BUCKETFS_PORT,
    "EXASOL_BUCKETFS_USER": EXASOL_BUCKETFS_USER,
    "EXASOL_BUCKETFS_PASSWORD": EXASOL_BUCKETFS_PASSWORD,
    "EXASOL_BUCKETFS_USE_HTTPS": EXASOL_BUCKETFS_USE_HTTPS,
    "EXASOL_BUCKETFS_BUCKET": EXASOL_BUCKETFS_BUCKET,
    "EXASOL_BUCKETFS_PATH": EXASOL_BUCKETFS_PATH
}

Let's do predictions on the data we have in the ABALONE_TEST table. This table also includes a column with ground truth labels. We will use it to access the performance of our predictor. In the code below we will add the ROWID to the output columns (as required by the generic prediction UDF). This will allow us to link the result to the ground truth.

In [5]:
import pyexasol
from stopwatch import Stopwatch

LABEL_COLUMN = 'RINGS'
params['test_table'] = 'ABALONE_TEST'
params['model_path'] = f'{EXASOL_BUCKETFS_PATH}/abalone_svm_model.pkl'

stopwatch = Stopwatch()

with pyexasol.connect(dsn=EXASOL_EXTERNAL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True) as conn:
    # Get the list of feature columns
    sql = 'SELECT * FROM {schema!i}.{test_table!i} LIMIT 1'
    df_tmp = conn.export_to_pandas(query_or_table=sql, query_params=params)
    params['column_names'] = [f'[{c}]' for c in df_tmp.columns if c != LABEL_COLUMN]

    # Get the predictions for all rows in the TEST table calling the prediction UDF.
    # Provide the model path and the row id int the first two parameters.
    sql = f'SELECT {{schema!i}}.SKLEARN_PREDICT({{model_path!s}}, ROWID, {{column_names!r}}) ' \
        f'emits ([sample_id] DECIMAL(20,0), [{LABEL_COLUMN}] DOUBLE) FROM {{schema!i}}.{{test_table!i}}'
    df_pred = conn.export_to_pandas(query_or_table=sql, query_params=params)

print(f"Getting predictions took: {stopwatch}")
df_pred.head()

Getting predictions took: 2.87s


,sample_id,RINGS
0,18446744073709552175,7.955215
1,18446744073709551965,7.536855
2,18446744073709552082,7.962816
3,18446744073709552267,7.651000
4,18446744073709552084,9.214513


We are going to check the performance of our predictor by linking the results to the ground truth labels and computing some regression metrics. This should give us similar results to what we have seen in the [training notebook](sklearn_train_abalone.ipynb).

In [6]:
import pandas as pd
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Get the ground truth labels for the test set.
with pyexasol.connect(dsn=EXASOL_EXTERNAL_HOST, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True) as conn:
    sql = f'SELECT ROWID AS [sample_id], [{LABEL_COLUMN}] FROM {{schema!i}}.{{test_table!i}}'
    df_true = conn.export_to_pandas(query_or_table=sql, query_params=params)

# Merge predictions and the ground truth on the sample id.
df_res = pd.merge(left=df_true, right=df_pred, on='sample_id', suffixes=['_true', '_pred'])

print('Mean absolute error:', mean_absolute_error(df_res[f'{LABEL_COLUMN}_true'], df_res[f'{LABEL_COLUMN}_pred']))
print('Mean squared error:', mean_squared_error(df_res[f'{LABEL_COLUMN}_true'], df_res[f'{LABEL_COLUMN}_pred']))
print('Explained variance:', explained_variance_score(df_res[f'{LABEL_COLUMN}_true'], df_res[f'{LABEL_COLUMN}_pred']))

Mean absolute error: 1.5339997227381341
Mean squared error: 5.284490858004687
Explained variance: 0.57724006901576
